In [1]:
!pip3 install tkintermapview
!pip install tkcalendar

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import tkinter
import tkintermapview

In [ ]:
###My Modification of GUI


import customtkinter
from tkintermapview import TkinterMapView
from flask import render_template, flash, request
import logging, io, base64, os, datetime
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import xgboost as xgb
from tkcalendar import DateEntry

customtkinter.set_default_color_theme("green")


class App(customtkinter.CTk):

    APP_NAME = "Earthquake Prediction"
    WIDTH = 800
    HEIGHT = 500
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.title(App.APP_NAME)
        self.geometry(str(App.WIDTH) + "x" + str(App.HEIGHT))
        self.minsize(App.WIDTH, App.HEIGHT)

        self.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.bind("<Command-q>", self.on_closing)
        self.bind("<Command-w>", self.on_closing)
        self.createcommand('tk::mac::Quit', self.on_closing)

        self.marker_list = []


        self.grid_columnconfigure(0, weight=0)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self, width=150, corner_radius=0, fg_color=None)
        self.frame_left.grid(row=0, column=0, padx=0, pady=0, sticky="nsew")

        self.frame_right = customtkinter.CTkFrame(master=self, corner_radius=0)
        self.frame_right.grid(row=0, column=1, rowspan=1, pady=0, padx=0, sticky="nsew")

    
        
        self.frame_left.grid_rowconfigure(2, weight=1)
        
        self.map_label = customtkinter.CTkLabel(self.frame_left, text="Select Date", anchor="w")
        self.date_picker = DateEntry(master=self.frame_left, width=12, background='green', foreground='white', 
                                     borderwidth=4, year=2023, month=5, day=3,font=('Helvetica', 12))
        self.date_picker.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        self.date_picker.bind("<<DateEntrySelected>>", self.on_date_selected)
        
        self.button_4 = customtkinter.CTkButton(master=self.frame_left,
                                                text="Earthquake estimates",
                                                command=self.get_earth_quake_estimates)
        self.button_4.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        

        
        self.map_label = customtkinter.CTkLabel(self.frame_left, text="Tile Server:", anchor="w")
        self.map_label.grid(row=3, column=0, padx=(20, 20), pady=(20, 0))
        self.map_option_menu = customtkinter.CTkOptionMenu(self.frame_left, values=["OpenStreetMap", "Google normal", "Google satellite"],
                                                                       command=self.change_map)
        self.map_option_menu.grid(row=4, column=0, padx=(20, 20), pady=(10, 0))

        self.appearance_mode_label = customtkinter.CTkLabel(self.frame_left, text="Appearance Mode:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=(20, 20), pady=(20, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.frame_left, values=["Light", "Dark", "System"],
                                                                       command=self.change_appearance_mode)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=(20, 20), pady=(10, 20))


        self.frame_right.grid_rowconfigure(1, weight=1)
        self.frame_right.grid_rowconfigure(0, weight=0)
        self.frame_right.grid_columnconfigure(0, weight=1)
        self.frame_right.grid_columnconfigure(1, weight=0)
        self.frame_right.grid_columnconfigure(2, weight=1)

        self.map_widget = TkinterMapView(self.frame_right, corner_radius=0)
        self.map_widget.grid(row=1, rowspan=1, column=0, columnspan=3, sticky="nswe", padx=(0, 0), pady=(0, 0))

        self.entry = customtkinter.CTkEntry(master=self.frame_right,
                                            placeholder_text="type place")
        self.entry.grid(row=0, column=0, sticky="we", padx=(12, 0), pady=12)
        self.entry.bind("<Return>", self.search_event)

        self.button_5 = customtkinter.CTkButton(master=self.frame_right,
                                                text="Search",
                                                width=90,
                                                command=self.search_event)
        self.button_5.grid(row=0, column=1, sticky="w", padx=(12, 0), pady=12)

        self.map_widget.set_address("Asia")
        self.map_option_menu.set("Google normal")
        self.appearance_mode_optionemenu.set("Light")
        
        
        
    def on_date_selected(self, event=None):
        selected_date = self.date_picker.get_date()
        #print(f"The selected date is {selected_date}")
        return str(selected_date)
        
    def prepare_earthquake_data_and_model(self,days_out_to_predict = 7, max_depth=3, eta=0.1):
        # get latest data from USGS servers
        df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
        df = df.sort_values('time', ascending=True)
        df['date'] = df['time'].str[0:10]

        df = df[['date', 'latitude', 'longitude', 'depth', 'mag', 'place']]
        temp_df = df['place'].str.split(', ', expand=True) 
        df['place'] = temp_df[1]
        df = df[['date', 'latitude', 'longitude', 'depth', 'mag', 'place']]

        df_coords = df[['place', 'latitude', 'longitude']]
        df_coords = df_coords.groupby(['place'], as_index=False).mean()
        df_coords = df_coords[['place', 'latitude', 'longitude']]

        df = df[['date', 'depth', 'mag', 'place']]
        df = pd.merge(left=df, right=df_coords, how='inner', on=['place'])

        eq_data = []
        df_live = []
        for symbol in list(set(df['place'])):
            temp_df = df[df['place'] == symbol].copy()
            temp_df['depth_avg_22'] = temp_df['depth'].rolling(window=22,center=False).mean() 
            temp_df['depth_avg_15'] = temp_df['depth'].rolling(window=15,center=False).mean()
            temp_df['depth_avg_7'] = temp_df['depth'].rolling(window=7,center=False).mean()
            temp_df['mag_avg_22'] = temp_df['mag'].rolling(window=22,center=False).mean() 
            temp_df['mag_avg_15'] = temp_df['mag'].rolling(window=15,center=False).mean()
            temp_df['mag_avg_7'] = temp_df['mag'].rolling(window=7,center=False).mean()
            temp_df.loc[:, 'mag_outcome'] = temp_df.loc[:, 'mag_avg_7'].shift(days_out_to_predict * -1)

            df_live.append(temp_df.tail(days_out_to_predict))

            eq_data.append(temp_df)

        df = pd.concat(eq_data)

        df = df[np.isfinite(df['depth_avg_22'])]
        df = df[np.isfinite(df['mag_avg_22'])]
        df = df[np.isfinite(df['mag_outcome'])]

        df['mag_outcome'] = np.where(df['mag_outcome'] > 2.5, 1,0)

        df = df[['date',
             'latitude',
             'longitude',
             'depth_avg_22',
             'depth_avg_15',
             'depth_avg_7',
             'mag_avg_22', 
             'mag_avg_15',
             'mag_avg_7',
             'mag_outcome']]

        df_live = pd.concat(df_live)
        df_live = df_live[np.isfinite(df_live['mag_avg_22'])]

        from sklearn.model_selection import train_test_split
        features = [f for f in list(df) if f not in ['date', 'mag_outcome', 'latitude',
         'longitude']]

        X_train, X_test, y_train, y_test = train_test_split(df[features],
                         df['mag_outcome'], test_size=0.3, random_state=42)

        dtrain = xgb.DMatrix(X_train[features], label=y_train)
        dtest = xgb.DMatrix(X_test[features], label=y_test)

        param = {
            'booster': 'gbtree',
            'eval_metric': 'auc',
            'max_depth': max_depth,  
            'eta': eta,  
            }  

        num_round = 1000    
        early_stopping_rounds=30
        xgb_model = xgb.train(param, dtrain, num_round) 


        dlive = xgb.DMatrix(df_live[features])  
        preds = xgb_model.predict(dlive)


        df_live = df_live[['date', 'place', 'latitude', 'longitude']]

        df_live = df_live.assign(preds=pd.Series(preds).values)


        df_live = df_live.groupby(['date', 'place'], as_index=False).mean()

        df_live['date']= pd.to_datetime(df_live['date'],format='%Y-%m-%d') 
        df_live['date'] = df_live['date'] + pd.to_timedelta(days_out_to_predict,unit='d')
        print(df_live)
        return(df_live)
    
    
    
    def get_earth_quake_estimates(self):
        
        desired_date=self.on_date_selected()
        print("The desired date is", desired_date)
        print("Model is Estimating")
        df_live=self.prepare_earthquake_data_and_model(7, 3, 0.1)
    
        from datetime import datetime
        live_set_tmp = df_live[df_live['date'] == desired_date]

    
        LatLngString = ''
        for marker in self.marker_list:
            self.map_widget.delete(marker)
        self.marker_list.clear()
        if (len(live_set_tmp) > 0):
            for lat, lon, pred in zip(live_set_tmp['latitude'], live_set_tmp['longitude'], live_set_tmp['preds']): 
                if(pred>0.3):
                    self.marker_list.append(self.map_widget.set_marker(lat, lon))
        return(LatLngString)
    
    
    
    
    

    def search_event(self, event=None):
        self.map_widget.set_address(self.entry.get())
    def change_appearance_mode(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_map(self, new_map: str):
        if new_map == "OpenStreetMap":
            self.map_widget.set_tile_server("https://a.tile.openstreetmap.org/{z}/{x}/{y}.png")
        elif new_map == "Google normal":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=m&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)
        elif new_map == "Google satellite":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)

    def on_closing(self, event=0):
        self.destroy()

    def start(self):
        self.mainloop()


if __name__ == "__main__":
    app = App()
    app.start()

The desired date is 2023-05-18
Model is Estimating
         date             place   latitude   longitude     preds
0  2023-05-10            Oregon  44.105931 -122.358435  0.079933
1  2023-05-11       Philippines  11.748437  123.984174  0.997369
2  2023-05-12             Chile -26.786409  -70.113081  0.999738
3  2023-05-12  Papua New Guinea  -5.251480  147.951535  0.999302
4  2023-05-12       Philippines  11.748437  123.984174  0.998442
..        ...               ...        ...         ...       ...
67 2023-05-19            Nevada  37.960989 -117.554600  0.000940
68 2023-05-19       Philippines  11.748437  123.984174  1.010923
69 2023-05-19       Puerto Rico  18.095924  -66.858990  0.399345
70 2023-05-19             Texas  31.194938 -102.822020  0.308430
71 2023-05-19            Turkey  37.962850   34.902295  0.673247

[72 rows x 5 columns]
